# Structuring data

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from PIL import Image
%matplotlib inline
from glob import glob

In [6]:
female_path = glob("./data/crop/female_crop/*.png")
male_path = female = glob("./data/crop/male_crop/*.png")

In [8]:
path = female_path + male_path

['./data/crop/male_crop/male_5630.png',
 './data/crop/male_crop/male_4131.png',
 './data/crop/male_crop/male_4869.png',
 './data/crop/male_crop/male_6032.png',
 './data/crop/male_crop/male_2372.png',
 './data/crop/male_crop/male_6219.png',
 './data/crop/male_crop/male_1422.png',
 './data/crop/male_crop/male_1461.png',
 './data/crop/male_crop/male_1410.png',
 './data/crop/male_crop/male_3207.png',
 './data/crop/male_crop/male_4147.png',
 './data/crop/male_crop/male_1529.png',
 './data/crop/male_crop/male_3519.png',
 './data/crop/male_crop/male_2276.png',
 './data/crop/male_crop/male_1081.png',
 './data/crop/male_crop/male_4445.png',
 './data/crop/male_crop/male_430.png',
 './data/crop/male_crop/male_6993.png',
 './data/crop/male_crop/male_605.png',
 './data/crop/male_crop/male_4610.png',
 './data/crop/male_crop/male_6290.png',
 './data/crop/male_crop/male_3497.png',
 './data/crop/male_crop/male_1744.png',
 './data/crop/male_crop/male_4094.png',
 './data/crop/male_crop/male_5936.png',
 '

Gets size of image.

In [13]:
def get_size(path):
    img = Image.open(path)
    return img.size[0]

In [14]:
get_size(path[0])

122

Makes dataframe with pandas.

In [16]:
dataframe = pd.DataFrame(data= path, columns=["path"])
dataframe.head()
dataframe.shape

(6059, 1)

In [17]:
dataframe["size"] = dataframe["path"].apply(get_size)
dataframe.head()

,path,size
0,./data/crop/female_crop/female_6653.png,122
1,./data/crop/female_crop/female_3869.png,182
2,./data/crop/female_crop/female_396.png,182
3,./data/crop/female_crop/female_4276.png,122
4,./data/crop/female_crop/female_5498.png,273
